### Run Panel Regression using 1971-2016 data

In [1]:
# Import libraries
from library.common import Database
import pathlib

import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import altair as alt
import warnings
warnings.filterwarnings("ignore")
import statsmodels.api as sm
from linearmodels import PanelOLS
from linearmodels import RandomEffects

# Display all columns of pandas dataframes
pd.set_option('display.max_columns', None)

db = Database()
write_to_db = True

#Get the path to raw data files
#path = pathlib.Path.cwd().parent.joinpath(path_to_data)
#path

Read the master dataset

In [2]:
data2 = pd.read_sql('master_dataset', db.config)
data2

,iso_code,country,year,co2,consumption_co2,co2_growth_prct,co2_growth_abs,trade_co2,co2_per_capita,consumption_co2_per_capita,share_global_co2,cumulative_co2,share_global_cumulative_co2,co2_per_gdp,consumption_co2_per_gdp,co2_per_unit_energy,coal_co2,cement_co2,flaring_co2,gas_co2,oil_co2,other_industry_co2,cement_co2_per_capita,coal_co2_per_capita,flaring_co2_per_capita,gas_co2_per_capita,oil_co2_per_capita,other_co2_per_capita,trade_co2_share,share_global_cement_co2,share_global_coal_co2,share_global_flaring_co2,share_global_gas_co2,share_global_oil_co2,share_global_other_co2,cumulative_cement_co2,cumulative_coal_co2,cumulative_flaring_co2,cumulative_gas_co2,cumulative_oil_co2,cumulative_other_co2,share_global_cumulative_cement_co2,share_global_cumulative_coal_co2,share_global_cumulative_flaring_co2,share_global_cumulative_gas_co2,share_global_cumulative_oil_co2,share_global_cumulative_other_co2,total_ghg,ghg_per_capita,methane,methane_per_capita,nitrous_oxide,nitrous_oxide_per_capita,gdp,primary_energy_consumption,energy_per_capita,energy_per_gdp,population,current_gdp,constant_gdp,manufacturing_gdp,medium_to_high_tech_percent,export,import,real_gdp_growth_percent,urban_population_percent,merchandise_export,merchandise_import,manufacturer_export_share,manufacturer_export,co2_emission_electricity,co2_emission_building,co2_emission_manufacturing,co2_emission_other_fuel,co2_emission_fugitive,co2_emission_transport,co2_emission_energy_subtotal,co2_emission_bunkers,co2_emission_industrial_process,co2_emission_per_capita,constant_gdp_per_capita,manufacturing_percent,medium_to_high_tech_gdp,co2_emission_per_constant_gdp,trade_openness,share_of_merchandise_export,share_of_merchandise_import,industrial_gdp,co2_country_share_percent,manufacturing_country_share_percent,iea_primary_energy_consumption,renewable_energy_consumption,coal_consumption,oil_consumption,total_electricity_production,electricity_production_from_renewable,primary_energy_consumption_per_capita,fossil_energy_consumption_share,renewable_electricity_production_share,energy_intensity,renewable_energy_consumption_share,percent_of_environment_patent,region,income_group
0,AFG,Afghanistan,1949,0.015,NaN,NaN,NaN,NaN,0.002,NaN,NaN,0.015,NaN,NaN,NaN,NaN,0.015,NaN,NaN,NaN,NaN,NaN,NaN,0.002,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.015,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,7624058.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
1,AFG,Afghanistan,1950,0.084,NaN,475.0,0.070,NaN,0.011,NaN,NaN,0.099,NaN,0.009,NaN,NaN,0.021,NaN,NaN,NaN,0.063,NaN,NaN,0.003,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.036,NaN,NaN,0.063,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.421400e+09,NaN,NaN,NaN,7752117.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
2,AFG,Afghanistan,1951,0.092,NaN,8.7,0.007,NaN,0.012,NaN,NaN,0.191,NaN,0.010,NaN,NaN,0.026,NaN,NaN,NaN,0.066,NaN,NaN,0.003,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.061,NaN,NaN,0.129,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,9.692280e+09,NaN,NaN,NaN,7840151.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
3,AFG,Afghanistan,1952,0.092,NaN,NaN,NaN,NaN,0.012,NaN,NaN,0.282,NaN,0.009,NaN,NaN,0.032,NaN,NaN,NaN,0.060,NaN,NaN,0.004,NaN,NaN,0.008,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.093,NaN,NaN,0.189,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.001732e+10,NaN,NaN,NaN,7935996.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,No

In [3]:
col_use = ['year','iso_code','region','income_group','co2','co2_emission_per_capita','co2_emission_per_constant_gdp',
           'population','urban_population_percent','constant_gdp_per_capita','manufacturing_percent','manufacturing_country_share_percent',
           'trade_openness','primary_energy_consumption_per_capita','renewable_energy_consumption_share','percent_of_environment_patent','energy_intensity',
          ]
data2=data2[col_use]

In [4]:
# Clusters obtained from machine learning algorithm 
country_gp0=['AFG', 'ALB', 'AND', 'AGO', 'ATG', 'ARM', 'ABW', 'BHS', 'BHR', 'BRB', 'BLZ', 'BEN', 'BMU', 'BTN', 'BOL', 'BIH', 
             'BWA', 'VGB', 'BRN', 'BFA', 'BDI', 'KHM', 'CMR', 'CPV', 'CAF', 'TCD', 'COM', 'COG', 'CRI', 'CIV', 'CUB', 'CUW', 
             'COD', 'DJI', 'DMA', 'DOM', 'SLV', 'GNQ', 'ERI', 'SWZ', 'ETH', 'FRO', 'FJI', 'PYF', 'GAB', 'GMB', 'GEO', 'GHA', 
             'GRL', 'GRD', 'GTM', 'GIN', 'GNB', 'GUY', 'HTI', 'HND', 'ISL', 'ATA', 'JAM', 'JOR', 'KEN', 'KIR', 'KGZ', 'LAO', 
             'LBN', 'LSO', 'LBR', 'LBY', 'LIE', 'MDG', 'MWI', 'MDV', 'MLI', 'MLT', 'MHL', 'MRT', 'MUS', 'MDA', 'MNG', 'MNE', 
             'MOZ', 'MMR', 'NAM', 'NRU', 'NPL', 'CXR', 'NCL', 'NIC', 'NER', 'NGA', 'PRK', 'PLW', 'PSE', 'PAN', 'PNG', 'PRY', 
             'PRI', 'RWA', 'KNA', 'LCA', 'VCT', 'WSM', 'STP', 'SEN', 'SRB', 'SYC', 'SLE', 'SXM', 'SLB', 'SOM', 'SSD', 'LKA', 
             'SDN', 'SUR', 'SYR', 'TJK', 'TZA', 'TLS', 'TGO', 'TON', 'TUN', 'TCA', 'TUV', 'UGA', 'URY', 'VUT', 'YEM', 'ZMB', 
             'ZWE', 'AIA', 'BES', 'COK', 'GUF', 'GLP', 'MTQ', 'MYT', 'MSR', 'NIU', 'SHN', 'SPM', 'TWN', 'ASM', 'CHI', 'CYM', 
             'FSM', 'GIB', 'GUM', 'IMN', 'MAC', 'MAF', 'MCO', 'MNP', 'SMR', 'VIR', 'XKX'] # Cluster 0
country_gp1=['CHN','USA'] # Cluster 1
country_gp2=['DZA', 'ARG', 'AUS', 'AUT', 'AZE', 'BGD', 'BLR', 'BEL', 'BRA', 'BGR', 'CAN', 'CHL', 'COL', 'HRV', 'CYP', 'CZE', 
             'DNK', 'ECU', 'EGY', 'EST', 'FIN', 'FRA', 'DEU', 'GRC', 'HKG', 'HUN', 'IND', 'IDN', 'IRN', 'IRQ', 'IRL', 'ISR', 
             'ITA', 'JPN', 'KAZ', 'KWT', 'LVA', 'LTU', 'LUX', 'MYS', 'MEX', 'MAR', 'NLD', 'NZL', 'MKD', 'NOR', 'OMN', 'PAK', 
             'PER', 'PHL', 'POL', 'PRT', 'QAT', 'ROU', 'RUS', 'SAU', 'SGP', 'SVK', 'SVN', 'ZAF', 'KOR', 'ESP', 'SWE', 'CHE', 
             'THA', 'TTO', 'TUR', 'TKM', 'UKR', 'ARE', 'GBR', 'UZB', 'VEN', 'VNM'] # Cluster 2

Compile the true total CO2 emission from 2017 to 2020 by clusters

In [5]:
# Compile true co2 value in 2017-2020
df_true_2016=data2[data2['year']==2016]
df_true_2017=data2[data2['year']==2017]
df_true_2018=data2[data2['year']==2018]
df_true_2019=data2[data2['year']==2019]
df_true_2020=data2[data2['year']==2020]
df_true_2016_world=df_true_2016[df_true_2016['iso_code']=='WLD']
df_true_2017_world=df_true_2017[df_true_2017['iso_code']=='WLD']
df_true_2018_world=df_true_2018[df_true_2018['iso_code']=='WLD']
df_true_2019_world=df_true_2019[df_true_2019['iso_code']=='WLD']
df_true_2020_world=df_true_2020[df_true_2020['iso_code']=='WLD']
df_true_2016=df_true_2016[df_true_2016['iso_code']!='WLD']
df_true_2017=df_true_2017[df_true_2017['iso_code']!='WLD']
df_true_2018=df_true_2018[df_true_2018['iso_code']!='WLD']
df_true_2019=df_true_2019[df_true_2019['iso_code']!='WLD']
df_true_2020=df_true_2020[df_true_2020['iso_code']!='WLD']
df_true_2016['cluster']=np.where(df_true_2016['iso_code'].isin(country_gp0),'cluster 0',np.nan)
df_true_2016['cluster']=np.where(df_true_2016['iso_code'].isin(country_gp1),'cluster 1',df_true_2016['cluster'])
df_true_2016['cluster']=np.where(df_true_2016['iso_code'].isin(country_gp2),'cluster 2',df_true_2016['cluster'])
df_true_2017['cluster']=np.where(df_true_2017['iso_code'].isin(country_gp0),'cluster 0',np.nan)
df_true_2017['cluster']=np.where(df_true_2017['iso_code'].isin(country_gp1),'cluster 1',df_true_2017['cluster'])
df_true_2017['cluster']=np.where(df_true_2017['iso_code'].isin(country_gp2),'cluster 2',df_true_2017['cluster'])
df_true_2018['cluster']=np.where(df_true_2018['iso_code'].isin(country_gp0),'cluster 0',np.nan)
df_true_2018['cluster']=np.where(df_true_2018['iso_code'].isin(country_gp1),'cluster 1',df_true_2018['cluster'])
df_true_2018['cluster']=np.where(df_true_2018['iso_code'].isin(country_gp2),'cluster 2',df_true_2018['cluster'])
df_true_2019['cluster']=np.where(df_true_2019['iso_code'].isin(country_gp0),'cluster 0',np.nan)
df_true_2019['cluster']=np.where(df_true_2019['iso_code'].isin(country_gp1),'cluster 1',df_true_2019['cluster'])
df_true_2019['cluster']=np.where(df_true_2019['iso_code'].isin(country_gp2),'cluster 2',df_true_2019['cluster'])
df_true_2020['cluster']=np.where(df_true_2020['iso_code'].isin(country_gp0),'cluster 0',np.nan)
df_true_2020['cluster']=np.where(df_true_2020['iso_code'].isin(country_gp1),'cluster 1',df_true_2020['cluster'])
df_true_2020['cluster']=np.where(df_true_2020['iso_code'].isin(country_gp2),'cluster 2',df_true_2020['cluster'])

df_true_2017 = df_true_2017.groupby(['cluster']).agg({'co2':'sum'}).reset_index()
df_true_2017_total=df_true_2017['co2'].sum()
dict_true_2017={'cluster':['all'],'co2':[df_true_2017_total]}
total_true_2017_df=pd.DataFrame.from_dict(dict_true_2017)
df_true_2017 = df_true_2017.append(total_true_2017_df)
df_true_2017['year']=2017
df_true_2017=df_true_2017[['year','cluster','co2']]

df_true_2018 = df_true_2018.groupby(['cluster']).agg({'co2':'sum'}).reset_index()
df_true_2018_total=df_true_2018['co2'].sum()
dict_true_2018={'cluster':['all'],'co2':[df_true_2018_total]}
total_true_2018_df=pd.DataFrame.from_dict(dict_true_2018)
df_true_2018 = df_true_2018.append(total_true_2018_df)
df_true_2018['year']=2018
df_true_2018=df_true_2018[['year','cluster','co2']]   

df_true_2019 = df_true_2019.groupby(['cluster']).agg({'co2':'sum'}).reset_index()
df_true_2019_total=df_true_2019['co2'].sum()
dict_true_2019={'cluster':['all'],'co2':[df_true_2019_total]}
total_true_2019_df=pd.DataFrame.from_dict(dict_true_2019)
df_true_2019 = df_true_2019.append(total_true_2019_df)
df_true_2019['year']=2019
df_true_2019=df_true_2019[['year','cluster','co2']]

df_true_2020 = df_true_2020.groupby(['cluster']).agg({'co2':'sum'}).reset_index()
df_true_2020_total=df_true_2020['co2'].sum()
dict_true_2020={'cluster':['all'],'co2':[df_true_2020_total]}
total_true_2020_df=pd.DataFrame.from_dict(dict_true_2020)
df_true_2020 = df_true_2020.append(total_true_2020_df)
df_true_2020['year']=2020
df_true_2020=df_true_2020[['year','cluster','co2']]

df_true=df_true_2017.append(df_true_2018)
df_true=df_true.append(df_true_2019)
df_true=df_true.append(df_true_2020)

In [6]:
diff_lst=['MTQ','GUF','MSR','COK','BES','ATA','SHN','TWN','SPM','MYT','AIA','CXR','GLP','NIU']
data2=data2[data2['year']>=1960]
data2_world=data2[data2['iso_code']=='WLD']
data2=data2[data2['iso_code']!='WLD']
data2=data2[~data2['iso_code'].isin(diff_lst)]
data2=data2.sort_values(["year", "iso_code"], ascending = (True, True))
data2=data2.append(data2_world).reset_index()
data2.drop(['index'],axis=1,inplace=True)
data2['co2']=data2['co2']*1000
data2['constant_gdp']=data2['constant_gdp_per_capita']*data2['population']
data2['primary_energy_consumption']=data2['primary_energy_consumption_per_capita']*data2['population']
data2['renewable_energy_consumption_per_capita']=(data2['primary_energy_consumption']*0.000000041868*(data2['renewable_energy_consumption_share']/100))/data2['population']

In [7]:
# Subset observations from 1971 for model fitting because we do not have 1960-1970 energy consumption data
analysis_df=data2[(data2['year']>=1971)&(data2['iso_code']!='WLD')]
analysis_df['country_gp0']=np.where(analysis_df['iso_code'].isin(country_gp0),1,0)
analysis_df['country_gp1']=np.where(analysis_df['iso_code'].isin(country_gp1),1,0)
analysis_df['country_gp2']=np.where(analysis_df['iso_code'].isin(country_gp2),1,0)

# Take log on independent and dependent variables
analysis_df['log_total_co2']=np.log(analysis_df['co2'])
analysis_df['log_co2_per_capita']=np.log(analysis_df['co2_emission_per_capita'])
analysis_df['log_co2_intensity']=np.log(analysis_df['co2_emission_per_constant_gdp'])
analysis_df['log_population']=np.log(analysis_df['population'])
analysis_df['log_urban_percent']=np.log(analysis_df['urban_population_percent'])
analysis_df['log_gdp_per_capita']=np.log(analysis_df['constant_gdp_per_capita'])
analysis_df['log_gdp_per_capita_sq']=np.log(analysis_df['constant_gdp_per_capita'])*np.log(analysis_df['constant_gdp_per_capita'])
analysis_df['log_manufacturing_share']=np.log(analysis_df['manufacturing_percent'])
analysis_df['log_trade']=np.log(analysis_df['trade_openness'])
analysis_df['log_energy_intensity']=np.log(analysis_df['energy_intensity'])
analysis_df['log_renewable_share']=np.log(analysis_df['renewable_energy_consumption_share'])
analysis_df['log_renewable_share_sq']=np.log(analysis_df['renewable_energy_consumption_share'])*np.log(analysis_df['renewable_energy_consumption_share'])
analysis_df['log_patent_percent']=np.log(analysis_df['percent_of_environment_patent'])

In [8]:
# Subset training dataframe, covering only 1971 - 2016 data
analysis_2000_df=analysis_df[(analysis_df['year']>=1971)&(analysis_df['year']<=2016)&(analysis_df['iso_code']!='WLD')]
analysis_2000_df2 = analysis_2000_df.copy(deep=True)
analysis_2000_df = analysis_2000_df.dropna(subset=['log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                                                   'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
                                                  ])

# Break up dataframe into three clusters
analysis_2000_df_cluster0 = analysis_2000_df[analysis_2000_df['country_gp0']==1]
analysis_2000_df_cluster1 = analysis_2000_df[analysis_2000_df['country_gp1']==1]
analysis_2000_df_cluster2 = analysis_2000_df[analysis_2000_df['country_gp2']==1]
analysis_2000_df_cluster12 = analysis_2000_df[(analysis_2000_df['country_gp1']==1)|(analysis_2000_df['country_gp2']==1)]

# Create a dataframe copy
analysis_2000_df2_cluster0 = analysis_2000_df_cluster0.copy(deep=True)
analysis_2000_df2_cluster1 = analysis_2000_df_cluster1.copy(deep=True)
analysis_2000_df2_cluster2 = analysis_2000_df_cluster2.copy(deep=True)
analysis_2000_df2_cluster12 = analysis_2000_df_cluster12.copy(deep=True)

In [9]:
# Get the entity and time parameters as well - all clusters
co2_pc_exog_vars_hand=['iso_code','year','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                       'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq'
                      ]
features_use=['iso_code','year','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
              'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
              'log_total_co2','log_co2_per_capita','log_co2_intensity']
predict_hand_df=analysis_2000_df2.copy(deep=True)
predict_hand_df=predict_hand_df.dropna(subset=['log_total_co2','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                              'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
                              ])
exog_hand=predict_hand_df[co2_pc_exog_vars_hand]
country_lst=list(exog_hand['iso_code'].unique())[:]

for i in range(1971,2017):
    exog_hand[i]=np.where(exog_hand['year']==i,1,0)
for country in country_lst:
    exog_hand[country]=np.where(exog_hand['iso_code']==country,1,0)
exog_hand.columns=exog_hand.columns.astype(str)    
exog_hand.drop(['year'],axis=1,inplace=True)
exog_hand2=exog_hand.copy(deep=True)
exog_hand.drop(['iso_code'],axis=1,inplace=True)
mod_2000_co2_pc = sm.OLS(predict_hand_df['log_total_co2'],sm.add_constant(exog_hand).astype(float))
mod_2000_co2_pc_res = mod_2000_co2_pc.fit(cov_type='HC3')

### In sample prediction

Predict total CO2 emission in 2017-2020, using the actual values of driving factors and the model parameters obtained from fitting the training dataset

In [10]:
# Prepare the matrix X to derive the predicted values
analysis_2000_test_df=analysis_df[(analysis_df['year']>=1971)&(analysis_df['year']<=2020)&(analysis_df['iso_code']!='WLD')]
analysis_2000_test_df = analysis_2000_test_df.dropna(subset=['log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                                                   'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
                                                  ])
exog_test=analysis_2000_test_df[co2_pc_exog_vars_hand]
country_lst=list(exog_test['iso_code'].unique())[:]

for country in country_lst:
    exog_test[country]=np.where(exog_test['iso_code']==country,1,0)
exog_test.columns=exog_test.columns.astype(str)  
exog_test['cluster']=np.where(exog_test['iso_code'].isin(country_gp0),'cluster 0',0)
exog_test['cluster']=np.where(exog_test['iso_code'].isin(country_gp1),'cluster 1',exog_test['cluster'])
exog_test['cluster']=np.where(exog_test['iso_code'].isin(country_gp2),'cluster 2',exog_test['cluster'])
exog_test1=exog_test[exog_test['year']==2017] # Get 2017 data as testing data
exog_test2=exog_test[exog_test['year']==2018] # Get 2018 data as testing data
exog_test3=exog_test[exog_test['year']==2019] # Get 2019 data as testing data
exog_test4=exog_test[exog_test['year']==2020] # Get 2020 data as testing data
predict_2017_result_df1=exog_test1[['iso_code','cluster']].reset_index()
predict_2017_result_df1.drop(['index'],axis=1,inplace=True)
predict_2018_result_df1=exog_test2[['iso_code','cluster']].reset_index()
predict_2018_result_df1.drop(['index'],axis=1,inplace=True)
predict_2019_result_df1=exog_test3[['iso_code','cluster']].reset_index()
predict_2019_result_df1.drop(['index'],axis=1,inplace=True)
predict_2020_result_df1=exog_test4[['iso_code','cluster']].reset_index()
predict_2020_result_df1.drop(['index'],axis=1,inplace=True)
exog_test1.drop(['iso_code','year','cluster'],axis=1,inplace=True)
exog_test2.drop(['iso_code','year','cluster'],axis=1,inplace=True)
exog_test3.drop(['iso_code','year','cluster'],axis=1,inplace=True)
exog_test4.drop(['iso_code','year','cluster'],axis=1,inplace=True)
exog_test1['2017']=1
exog_test1['constant']=1
exog_test2['2018']=1
exog_test2['constant']=1
exog_test3['2019']=1
exog_test3['constant']=1
exog_test4['2020']=1
exog_test4['constant']=1
exog_test1_array = exog_test1.to_numpy()
exog_test2_array = exog_test2.to_numpy()
exog_test3_array = exog_test3.to_numpy()
exog_test4_array = exog_test4.to_numpy()

In [11]:
# Prepare beta to derive the predicted value
sum_param_2025=0
for i in range(46):
    sum_param_2025=sum_param_2025+(mod_2000_co2_pc_res.params[[i+10]][0])
param_2025=sum_param_2025/46
beta_list=[]
for i in range(9):
    beta_list.append(mod_2000_co2_pc_res.params[[i+1]][0])
for i in range(160):
    beta_list.append(mod_2000_co2_pc_res.params[[i+56]][0])
beta_list.append(param_2025)
beta_list.append(mod_2000_co2_pc_res.params[[0]][0])
beta_array=np.array(beta_list)
#beta_array.shape

In [12]:
# y = Xb Calculate predicted log total CO2 by country
y1_hat_array=np.exp(np.matmul(exog_test1_array,beta_array))
y2_hat_array=np.exp(np.matmul(exog_test2_array,beta_array))
y3_hat_array=np.exp(np.matmul(exog_test3_array,beta_array))
y4_hat_array=np.exp(np.matmul(exog_test4_array,beta_array))
#y_hat_array

In [13]:
# Predict 2017 total CO2 emission using the panel regression model (using individual cluster figures) - total
#(Note: The figure is divided by 1000 to convert the unit from thousand tonne to million tonne for comparison with the time series forecast)
predict_2017_result_df2=pd.DataFrame(y1_hat_array,columns=['predict_total_co2'])
predict_2017_result_df = pd.concat([predict_2017_result_df1.reset_index(drop=True),predict_2017_result_df2.reset_index(drop=True)], axis=1)
predict_2017_total_co2 = (predict_2017_result_df['predict_total_co2'].sum())/1000

In [14]:
# Predict 2018 total CO2 emission using the panel regression model (using individual cluster figures) - total
#(Note: The figure is divided by 1000 to convert the unit from thousand tonne to million tonne for comparison with the time series forecast)
predict_2018_result_df2=pd.DataFrame(y2_hat_array,columns=['predict_total_co2'])
predict_2018_result_df = pd.concat([predict_2018_result_df1.reset_index(drop=True),predict_2018_result_df2.reset_index(drop=True)], axis=1)
predict_2018_total_co2 = (predict_2018_result_df['predict_total_co2'].sum())/1000

In [15]:
# Predict 2019 total CO2 emission using the panel regression model (using individual cluster figures) - total
#(Note: The figure is divided by 1000 to convert the unit from thousand tonne to million tonne for comparison with the time series forecast)
predict_2019_result_df2=pd.DataFrame(y3_hat_array,columns=['predict_total_co2'])
predict_2019_result_df = pd.concat([predict_2019_result_df1.reset_index(drop=True),predict_2019_result_df2.reset_index(drop=True)], axis=1)
predict_2019_total_co2 = (predict_2019_result_df['predict_total_co2'].sum())/1000

In [16]:
# Predict 2020 total CO2 emission using the panel regression model (using individual cluster figures) - total
#(Note: The figure is divided by 1000 to convert the unit from thousand tonne to million tonne for comparison with the time series forecast)
predict_2020_result_df2=pd.DataFrame(y4_hat_array,columns=['predict_total_co2'])
predict_2020_result_df = pd.concat([predict_2020_result_df1.reset_index(drop=True),predict_2020_result_df2.reset_index(drop=True)], axis=1)
predict_2020_total_co2 = (predict_2020_result_df['predict_total_co2'].sum())/1000

In [17]:
# Predict 2017 total CO2 emission using the panel regression model (using individual cluster figures) - individual cluster
predict_2017_cluster_co2 = predict_2017_result_df.groupby(['cluster']).agg({'predict_total_co2':'sum'})
predict_2017_cluster_co2['predict_total_co2']=predict_2017_cluster_co2['predict_total_co2']/1000
predict_2017_cluster_co2=predict_2017_cluster_co2.reset_index()
dict_2017={'cluster':['all'],'predict_total_co2':[predict_2017_total_co2]}
total_2017_df=pd.DataFrame.from_dict(dict_2017)
predict_2017_cluster_co2=predict_2017_cluster_co2.append(total_2017_df)
predict_2017_cluster_co2['year']=2017
predict_2017_cluster_co2=predict_2017_cluster_co2[['year','cluster','predict_total_co2']]

In [18]:
# Predict 2018 total CO2 emission using the panel regression model (using individual cluster figures) - individual cluster
predict_2018_cluster_co2 = predict_2018_result_df.groupby(['cluster']).agg({'predict_total_co2':'sum'})
predict_2018_cluster_co2['predict_total_co2']=predict_2018_cluster_co2['predict_total_co2']/1000
predict_2018_cluster_co2=predict_2018_cluster_co2.reset_index()
dict_2018={'cluster':['all'],'predict_total_co2':[predict_2018_total_co2]}
total_2018_df=pd.DataFrame.from_dict(dict_2018)
predict_2018_cluster_co2=predict_2018_cluster_co2.append(total_2018_df)
predict_2018_cluster_co2['year']=2018
predict_2018_cluster_co2=predict_2018_cluster_co2[['year','cluster','predict_total_co2']]

In [19]:
# Predict 2019 total CO2 emission using the panel regression model (using individual cluster figures) - individual cluster
predict_2019_cluster_co2 = predict_2019_result_df.groupby(['cluster']).agg({'predict_total_co2':'sum'})
predict_2019_cluster_co2['predict_total_co2']=predict_2019_cluster_co2['predict_total_co2']/1000
predict_2019_cluster_co2=predict_2019_cluster_co2.reset_index()
dict_2019={'cluster':['all'],'predict_total_co2':[predict_2019_total_co2]}
total_2019_df=pd.DataFrame.from_dict(dict_2019)
predict_2019_cluster_co2=predict_2019_cluster_co2.append(total_2019_df)
predict_2019_cluster_co2['year']=2019
predict_2019_cluster_co2=predict_2019_cluster_co2[['year','cluster','predict_total_co2']]

In [20]:
# Predict 2020 total CO2 emission using the panel regression model (using individual cluster figures) - individual cluster
predict_2020_cluster_co2 = predict_2020_result_df.groupby(['cluster']).agg({'predict_total_co2':'sum'})
predict_2020_cluster_co2['predict_total_co2']=predict_2020_cluster_co2['predict_total_co2']/1000
predict_2020_cluster_co2=predict_2020_cluster_co2.reset_index()
dict_2020={'cluster':['all'],'predict_total_co2':[predict_2020_total_co2]}
total_2020_df=pd.DataFrame.from_dict(dict_2020)
predict_2020_cluster_co2=predict_2020_cluster_co2.append(total_2020_df)
predict_2020_cluster_co2['year']=2020
predict_2020_cluster_co2=predict_2020_cluster_co2[['year','cluster','predict_total_co2']]

In [21]:
# Put all predicted results into a single dataframe
predict_test_cluster_co2=predict_2017_cluster_co2.append(predict_2018_cluster_co2)
predict_test_cluster_co2=predict_test_cluster_co2.append(predict_2019_cluster_co2)
predict_test_cluster_co2=predict_test_cluster_co2.append(predict_2020_cluster_co2)
predict_test_cluster_co2=pd.merge(predict_test_cluster_co2,df_true,how='left',on=['year','cluster'])
predict_test_cluster_co2

,year,cluster,predict_total_co2,co2
0,2017,cluster 0,892.477209,1345.464
1,2017,cluster 1,19252.443276,15128.210
2,2017,cluster 2,19714.499549,18205.638
3,2017,all,39859.420035,34679.312
4,2018,cluster 0,937.088876,1383.207
5,2018,cluster 1,19976.983043,15665.481
6,2018,cluster 2,20145.781957,18318.256
7,2018,all,41059.853875,35366.944
8,2019,cluster 0,942.161952,1482.379
9,2019,cluster 1,20541.530551,15745.805


In [22]:
# Predict global 2017-2020
beta_list_all_temp=[]
for i in range(10):
    beta_list_all_temp.append(mod_2000_co2_pc_res.params[[i]][0])
world_param_list=[]
for i in range(46):
    co2=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'co2']
    population=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'population']
    urban_population_percent=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'urban_population_percent']
    constant_gdp_per_capita=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'constant_gdp_per_capita']
    energy_intensity=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'energy_intensity']
    manufacturing_percent=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'manufacturing_percent']
    trade_openness=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'trade_openness']
    renewable_energy_share=data2.loc[data2.index[(data2['year']==i+1971)&(data2['iso_code']=='WLD')][0],'renewable_energy_consumption_share']
    ln_co2=np.log(co2)
    ln_population=np.log(population)
    ln_urban_population_percent=np.log(urban_population_percent)
    ln_constant_gdp_per_capita=np.log(constant_gdp_per_capita)
    ln_constant_gdp_per_capita_sq=np.log(constant_gdp_per_capita)**2
    ln_energy_intensity=np.log(energy_intensity)
    ln_manufacturing_percent=np.log(manufacturing_percent)
    ln_trade_openness=np.log(trade_openness)
    ln_renewable_energy_share=np.log(renewable_energy_share)
    ln_renewable_energy_share_sq=np.log(renewable_energy_share)**2
    beta_list_all_temp.append(mod_2000_co2_pc_res.params[[i+10]][0])
    X_all_temp_array=np.array([1,ln_population,ln_urban_population_percent,ln_constant_gdp_per_capita,ln_constant_gdp_per_capita_sq,ln_energy_intensity,ln_manufacturing_percent,ln_trade_openness,ln_renewable_energy_share,ln_renewable_energy_share_sq,1])
    beta_all_temp_array=np.array(beta_list_all_temp)
    world_param=ln_co2-np.matmul(np.transpose(X_all_temp_array),beta_all_temp_array)
    world_param_list.append(world_param)
    beta_list_all_temp=beta_list_all_temp[:-1]
world_param_cleaned_list = [x for x in world_param_list if str(x) != 'nan'] 
world_param=sum(world_param_cleaned_list)/len(world_param_cleaned_list)

In [23]:
beta_list_all=[]
for i in range(9):
    beta_list_all.append(mod_2000_co2_pc_res.params[[i+1]][0])
beta_list_all.append(param_2025)
beta_list_all.append(mod_2000_co2_pc_res.params[[0]][0])
beta_list_all.append(world_param)
beta_all_array=np.array(beta_list_all)

In [24]:
df_true_2016_world['log_population']=np.log(df_true_2016_world['population'])
df_true_2016_world['log_urban_percent']=np.log(df_true_2016_world['urban_population_percent'])
df_true_2016_world['log_gdp_per_capita']=np.log(df_true_2016_world['constant_gdp_per_capita'])
df_true_2016_world['log_gpd_per_capita_sq']=df_true_2016_world['log_gdp_per_capita']*df_true_2016_world['log_gdp_per_capita']
df_true_2016_world['log_energy_intensity']=np.log(df_true_2016_world['energy_intensity'])
df_true_2016_world['log_manufacturing_share']=np.log(df_true_2016_world['manufacturing_percent'])
df_true_2016_world['log_trade']=np.log(df_true_2016_world['trade_openness'])
df_true_2016_world['log_renewable_share']=np.log(df_true_2016_world['renewable_energy_consumption_share'])
df_true_2016_world['log_renewable_share_sq']=np.log(df_true_2016_world['renewable_energy_consumption_share'])*np.log(df_true_2016_world['renewable_energy_consumption_share'])

df_true_2017_world['log_population']=np.log(df_true_2017_world['population'])
df_true_2017_world['log_urban_percent']=np.log(df_true_2017_world['urban_population_percent'])
df_true_2017_world['log_gdp_per_capita']=np.log(df_true_2017_world['constant_gdp_per_capita'])
df_true_2017_world['log_gpd_per_capita_sq']=df_true_2017_world['log_gdp_per_capita']*df_true_2017_world['log_gdp_per_capita']
df_true_2017_world['log_energy_intensity']=np.log(df_true_2017_world['energy_intensity'])
df_true_2017_world['log_manufacturing_share']=np.log(df_true_2017_world['manufacturing_percent'])
df_true_2017_world['log_trade']=np.log(df_true_2017_world['trade_openness'])
df_true_2017_world['log_renewable_share']=np.log(df_true_2017_world['renewable_energy_consumption_share'])
df_true_2017_world['log_renewable_share_sq']=np.log(df_true_2017_world['renewable_energy_consumption_share'])*np.log(df_true_2017_world['renewable_energy_consumption_share'])
df_true_2017_world['2017']=1
df_true_2017_world['constant']=1
df_true_2017_world['world']=1

df_true_2018_world['log_population']=np.log(df_true_2018_world['population'])
df_true_2018_world['log_urban_percent']=np.log(df_true_2018_world['urban_population_percent'])
df_true_2018_world['log_gdp_per_capita']=np.log(df_true_2018_world['constant_gdp_per_capita'])
df_true_2018_world['log_gpd_per_capita_sq']=df_true_2018_world['log_gdp_per_capita']*df_true_2018_world['log_gdp_per_capita']
df_true_2018_world['log_energy_intensity']=np.log(df_true_2018_world['energy_intensity'])
df_true_2018_world['log_manufacturing_share']=np.log(df_true_2018_world['manufacturing_percent'])
df_true_2018_world['log_trade']=np.log(df_true_2018_world['trade_openness'])
df_true_2018_world['log_renewable_share']=np.log(df_true_2018_world['renewable_energy_consumption_share'])
df_true_2018_world['log_renewable_share_sq']=np.log(df_true_2018_world['renewable_energy_consumption_share'])*np.log(df_true_2018_world['renewable_energy_consumption_share'])
df_true_2018_world['2018']=1
df_true_2018_world['constant']=1
df_true_2018_world['world']=1

df_true_2019_world['log_population']=np.log(df_true_2019_world['population'])
df_true_2019_world['log_urban_percent']=np.log(df_true_2019_world['urban_population_percent'])
df_true_2019_world['log_gdp_per_capita']=np.log(df_true_2019_world['constant_gdp_per_capita'])
df_true_2019_world['log_gpd_per_capita_sq']=df_true_2019_world['log_gdp_per_capita']*df_true_2019_world['log_gdp_per_capita']
df_true_2019_world['log_energy_intensity']=np.log(df_true_2019_world['energy_intensity'])
df_true_2019_world['log_manufacturing_share']=np.log(df_true_2019_world['manufacturing_percent'])
df_true_2019_world['log_trade']=np.log(df_true_2019_world['trade_openness'])
df_true_2019_world['log_renewable_share']=np.log(df_true_2019_world['renewable_energy_consumption_share'])
df_true_2019_world['log_renewable_share_sq']=np.log(df_true_2019_world['renewable_energy_consumption_share'])*np.log(df_true_2019_world['renewable_energy_consumption_share'])
df_true_2019_world['2019']=1
df_true_2019_world['constant']=1
df_true_2019_world['world']=1

df_true_2020_world['log_population']=np.log(df_true_2020_world['population'])
df_true_2020_world['log_urban_percent']=np.log(df_true_2020_world['urban_population_percent'])
df_true_2020_world['log_gdp_per_capita']=np.log(df_true_2020_world['constant_gdp_per_capita'])
df_true_2020_world['log_gpd_per_capita_sq']=df_true_2020_world['log_gdp_per_capita']*df_true_2020_world['log_gdp_per_capita']
df_true_2020_world['log_energy_intensity']=np.log(df_true_2020_world['energy_intensity'])
df_true_2020_world['log_manufacturing_share']=np.log(df_true_2020_world['manufacturing_percent'])
df_true_2020_world['log_trade']=np.log(df_true_2020_world['trade_openness'])
df_true_2020_world['log_renewable_share']=df_true_2019_world.loc[20953,'log_renewable_share']
df_true_2020_world['log_renewable_share_sq']=df_true_2019_world.loc[20953,'log_renewable_share_sq']
df_true_2020_world['2020']=1
df_true_2020_world['constant']=1
df_true_2020_world['world']=1

X_all_2017=df_true_2017_world[['log_population','log_urban_percent','log_gdp_per_capita','log_gpd_per_capita_sq','log_energy_intensity','log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq','2017','constant','world']]
X_all_2017_array=np.array(X_all_2017)
X_all_2018=df_true_2018_world[['log_population','log_urban_percent','log_gdp_per_capita','log_gpd_per_capita_sq','log_energy_intensity','log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq','2018','constant','world']]
X_all_2018_array=np.array(X_all_2018)
X_all_2019=df_true_2019_world[['log_population','log_urban_percent','log_gdp_per_capita','log_gpd_per_capita_sq','log_energy_intensity','log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq','2019','constant','world']]
X_all_2019_array=np.array(X_all_2019)
X_all_2020=df_true_2020_world[['log_population','log_urban_percent','log_gdp_per_capita','log_gpd_per_capita_sq','log_energy_intensity','log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq','2020','constant','world']]
X_all_2020_array=np.array(X_all_2020)

In [25]:
# Predict 2017-2020 total CO2 emission using the panel regression model (using global CO2 emission figure)
y_hat_2017_all_array=np.exp(np.matmul(X_all_2017_array,beta_all_array))/1000
y_hat_2018_all_array=np.exp(np.matmul(X_all_2018_array,beta_all_array))/1000
y_hat_2019_all_array=np.exp(np.matmul(X_all_2019_array,beta_all_array))/1000
y_hat_2020_all_array=np.exp(np.matmul(X_all_2020_array,beta_all_array))/1000
predict_total_co2_global=[y_hat_2017_all_array[0],y_hat_2018_all_array[0],y_hat_2019_all_array[0],y_hat_2020_all_array[0]]
year_predict_total_global=[2017,2018,2019,2020]
total_co2_global=[df_true_2017_world.loc[20951,'co2'],df_true_2018_world.loc[20952,'co2'],df_true_2019_world.loc[20953,'co2'],df_true_2020_world.loc[20954,'co2']]
predict_test_global_co2={'year':year_predict_total_global,'predict_total_co2':predict_total_co2_global,'co2':total_co2_global}
predict_test_global_co2_df=pd.DataFrame.from_dict(predict_test_global_co2)
predict_test_global_co2_df

,year,predict_total_co2,co2
0,2017,41419.521643,35925.738
1,2018,42479.081443,36646.140
2,2019,43062.141679,36702.503
3,2020,41810.826522,34807.259


### Out sample prediction

Predict 2025 total CO2 emission, using the model parameters obtained by fitting the training dataset, then read in the 2025 timeseries forecasts of individual driving factors to obtain the forecast

In [26]:
population_growth_2016_2020=(df_true_2020_world.loc[20954,'population']/df_true_2016_world.loc[20950,'population'])
urban_percent_growth_2016_2020=(df_true_2020_world.loc[20954,'urban_population_percent']/df_true_2016_world.loc[20950,'urban_population_percent'])
gdp_per_capita_growth_2016_2020=(df_true_2020_world.loc[20954,'constant_gdp_per_capita']/df_true_2016_world.loc[20950,'constant_gdp_per_capita'])
energy_intensity_growth_2016_2020=(df_true_2020_world.loc[20954,'energy_intensity']/df_true_2016_world.loc[20950,'energy_intensity'])
manufacturing_percent_growth_2016_2020=(df_true_2020_world.loc[20954,'manufacturing_percent']/df_true_2016_world.loc[20950,'manufacturing_percent'])
trade_openness_growth_2016_2020=(df_true_2020_world.loc[20954,'trade_openness']/df_true_2016_world.loc[20950,'trade_openness'])
renewable_share_growth_2016_2020=(df_true_2019_world.loc[20953,'renewable_energy_consumption_share']/df_true_2016_world.loc[20950,'renewable_energy_consumption_share'])

In [27]:
# Read forecast value of independent variables obtained from time series analysis
# Convert the forecast pickle file into format in need

dirname = os.path.abspath('')
filename = os.path.join(dirname, 'data/processed/ts_forecast.pkl')
filename = filename.replace('\\','/')
filename = filename.replace('notebooks/','')

forecast_input_lst = pd.read_pickle(filename) 
c0_population_array=forecast_input_lst[14][2]
c0_gdp_pc_array=forecast_input_lst[4][2]
c0_ei_array=forecast_input_lst[21][2]
c0_manufacturing_percent_array=forecast_input_lst[7][2]
c0_trade_array=forecast_input_lst[0][2]
c0_renew_percent_array=forecast_input_lst[10][2]
c0_co2_array=forecast_input_lst[5][2]

c1_population_array=forecast_input_lst[18][2]
c1_gdp_pc_array=forecast_input_lst[9][2]
c1_ei_array=forecast_input_lst[15][2]
c1_manufacturing_percent_array=forecast_input_lst[13][2]
c1_trade_array=forecast_input_lst[27][2]
c1_renew_percent_array=forecast_input_lst[12][2]
c1_co2_array=forecast_input_lst[11][2]

c2_population_array=forecast_input_lst[25][2]
c2_gdp_pc_array=forecast_input_lst[17][2]
c2_ei_array=forecast_input_lst[8][2]
c2_manufacturing_percent_array=forecast_input_lst[1][2]
c2_trade_array=forecast_input_lst[23][2]
c2_renew_percent_array=forecast_input_lst[16][2]
c2_co2_array=forecast_input_lst[2][2]

all_population_array=forecast_input_lst[3][2]
all_gdp_pc_array=forecast_input_lst[20][2]
all_ei_array=forecast_input_lst[22][2]
all_manufacturing_percent_array=forecast_input_lst[19][2]
all_trade_array=forecast_input_lst[6][2]
all_renew_percent_array=forecast_input_lst[24][2]
all_co2_array=forecast_input_lst[26][2]

c0_population_df = pd.DataFrame(c0_population_array,columns=['year','population'])
c0_population_df['year'].astype(int)
c0_gdp_pc_df = pd.DataFrame(c0_gdp_pc_array,columns=['year','constant_gdp_per_capita'])
c0_gdp_pc_df['year'].astype(int)
c0_ei_df = pd.DataFrame(c0_ei_array,columns=['year','energy_intensity'])
c0_ei_df['year'].astype(int)
c0_manufacturing_percent_df = pd.DataFrame(c0_manufacturing_percent_array,columns=['year','manufacturing_percent'])
c0_manufacturing_percent_df['year'].astype(int)
c0_trade_df = pd.DataFrame(c0_trade_array,columns=['year','trade_openness'])
c0_trade_df['year'].astype(int)
c0_renew_percent_df = pd.DataFrame(c0_renew_percent_array,columns=['year','renewable_energy_consumption_share'])
c0_renew_percent_df['year'].astype(int)
c0_forecast_input_df=c0_population_df.merge(c0_gdp_pc_df,how='left',on=['year'])
c0_forecast_input_df=c0_forecast_input_df.merge(c0_ei_df,how='left',on=['year'])
c0_forecast_input_df=c0_forecast_input_df.merge(c0_manufacturing_percent_df,how='left',on=['year'])
c0_forecast_input_df=c0_forecast_input_df.merge(c0_trade_df,how='left',on=['year'])
c0_forecast_input_df=c0_forecast_input_df.merge(c0_renew_percent_df,how='left',on=['year'])
c0_forecast_input_df['cluster']='cluster 0'
c0_forecast_input_df.drop(['year'],axis=1,inplace=True)
c0_forecast_input_df['year']=[2021,2022,2023,2024,2025]
c0_forecast_input_df['population_lag4']=c0_forecast_input_df['population'].shift(4)
c0_forecast_input_df['constant_gdp_per_capita_lag4']=c0_forecast_input_df['constant_gdp_per_capita'].shift(4)
c0_forecast_input_df['energy_intensity_lag4']=c0_forecast_input_df['energy_intensity'].shift(4)
c0_forecast_input_df['manufacturing_percent_lag4']=c0_forecast_input_df['manufacturing_percent'].shift(4)
c0_forecast_input_df['trade_openness_lag4']=c0_forecast_input_df['trade_openness'].shift(4)
c0_forecast_input_df['renewable_energy_consumption_share_lag4']=c0_forecast_input_df['renewable_energy_consumption_share'].shift(4)
c0_forecast_input_df['population_growth']=c0_forecast_input_df['population']/c0_forecast_input_df['population_lag4']
c0_forecast_input_df['constant_gdp_per_capita_growth']=c0_forecast_input_df['constant_gdp_per_capita']/c0_forecast_input_df['constant_gdp_per_capita_lag4']
c0_forecast_input_df['energy_intensity_growth']=c0_forecast_input_df['energy_intensity']/c0_forecast_input_df['energy_intensity_lag4']
c0_forecast_input_df['manufacturing_percent_growth']=c0_forecast_input_df['manufacturing_percent']/c0_forecast_input_df['manufacturing_percent_lag4']
c0_forecast_input_df['trade_openness_growth']=c0_forecast_input_df['trade_openness']/c0_forecast_input_df['trade_openness_lag4']
c0_forecast_input_df['renewable_energy_consumption_share_growth']=c0_forecast_input_df['renewable_energy_consumption_share']/c0_forecast_input_df['renewable_energy_consumption_share_lag4']

c0_population_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'population_growth']
c0_constant_gdp_per_capita_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'constant_gdp_per_capita_growth']
c0_energy_intensity_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'energy_intensity_growth']
c0_manufacturing_percent_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'manufacturing_percent_growth']
c0_trade_openness_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'trade_openness_growth']
c0_renewable_energy_consumption_share_growth=c0_forecast_input_df.loc[c0_forecast_input_df.index[(c0_forecast_input_df['year']==2025)][0],'renewable_energy_consumption_share_growth']

c1_population_df = pd.DataFrame(c1_population_array,columns=['year','population'])
c1_population_df['year'].astype(int)
c1_gdp_pc_df = pd.DataFrame(c1_gdp_pc_array,columns=['year','constant_gdp_per_capita'])
c1_gdp_pc_df['year'].astype(int)
c1_ei_df = pd.DataFrame(c1_ei_array,columns=['year','energy_intensity'])
c1_ei_df['year'].astype(int)
c1_manufacturing_percent_df = pd.DataFrame(c1_manufacturing_percent_array,columns=['year','manufacturing_percent'])
c1_manufacturing_percent_df['year'].astype(int)
c1_trade_df = pd.DataFrame(c1_trade_array,columns=['year','trade_openness'])
c1_trade_df['year'].astype(int)
c1_renew_percent_df = pd.DataFrame(c1_renew_percent_array,columns=['year','renewable_energy_consumption_share'])
c1_renew_percent_df['year'].astype(int)
c1_forecast_input_df=c1_population_df.merge(c1_gdp_pc_df,how='left',on=['year'])
c1_forecast_input_df=c1_forecast_input_df.merge(c1_ei_df,how='left',on=['year'])
c1_forecast_input_df=c1_forecast_input_df.merge(c1_manufacturing_percent_df,how='left',on=['year'])
c1_forecast_input_df=c1_forecast_input_df.merge(c1_trade_df,how='left',on=['year'])
c1_forecast_input_df=c1_forecast_input_df.merge(c1_renew_percent_df,how='left',on=['year'])
c1_forecast_input_df['cluster']='cluster 1'
c1_forecast_input_df.drop(['year'],axis=1,inplace=True)
c1_forecast_input_df['year']=[2021,2022,2023,2024,2025]
c1_forecast_input_df['population_lag4']=c1_forecast_input_df['population'].shift(4)
c1_forecast_input_df['constant_gdp_per_capita_lag4']=c1_forecast_input_df['constant_gdp_per_capita'].shift(4)
c1_forecast_input_df['energy_intensity_lag4']=c1_forecast_input_df['energy_intensity'].shift(4)
c1_forecast_input_df['manufacturing_percent_lag4']=c1_forecast_input_df['manufacturing_percent'].shift(4)
c1_forecast_input_df['trade_openness_lag4']=c1_forecast_input_df['trade_openness'].shift(4)
c1_forecast_input_df['renewable_energy_consumption_share_lag4']=c1_forecast_input_df['renewable_energy_consumption_share'].shift(4)
c1_forecast_input_df['population_growth']=c1_forecast_input_df['population']/c1_forecast_input_df['population_lag4']
c1_forecast_input_df['constant_gdp_per_capita_growth']=c1_forecast_input_df['constant_gdp_per_capita']/c1_forecast_input_df['constant_gdp_per_capita_lag4']
c1_forecast_input_df['energy_intensity_growth']=c1_forecast_input_df['energy_intensity']/c1_forecast_input_df['energy_intensity_lag4']
c1_forecast_input_df['manufacturing_percent_growth']=c1_forecast_input_df['manufacturing_percent']/c1_forecast_input_df['manufacturing_percent_lag4']
c1_forecast_input_df['trade_openness_growth']=c1_forecast_input_df['trade_openness']/c1_forecast_input_df['trade_openness_lag4']
c1_forecast_input_df['renewable_energy_consumption_share_growth']=c1_forecast_input_df['renewable_energy_consumption_share']/c1_forecast_input_df['renewable_energy_consumption_share_lag4']

c1_population_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'population_growth']
c1_constant_gdp_per_capita_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'constant_gdp_per_capita_growth']
c1_energy_intensity_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'energy_intensity_growth']
c1_manufacturing_percent_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'manufacturing_percent_growth']
c1_trade_openness_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'trade_openness_growth']
c1_renewable_energy_consumption_share_growth=c1_forecast_input_df.loc[c1_forecast_input_df.index[(c1_forecast_input_df['year']==2025)][0],'renewable_energy_consumption_share_growth']

c2_population_df = pd.DataFrame(c2_population_array,columns=['year','population'])
c2_population_df['year'].astype(int)
c2_gdp_pc_df = pd.DataFrame(c2_gdp_pc_array,columns=['year','constant_gdp_per_capita'])
c2_gdp_pc_df['year'].astype(int)
c2_ei_df = pd.DataFrame(c2_ei_array,columns=['year','energy_intensity'])
c2_ei_df['year'].astype(int)
c2_manufacturing_percent_df = pd.DataFrame(c2_manufacturing_percent_array,columns=['year','manufacturing_percent'])
c2_manufacturing_percent_df['year'].astype(int)
c2_trade_df = pd.DataFrame(c2_trade_array,columns=['year','trade_openness'])
c2_trade_df['year'].astype(int)
c2_renew_percent_df = pd.DataFrame(c2_renew_percent_array,columns=['year','renewable_energy_consumption_share'])
c2_renew_percent_df['year'].astype(int)
c2_forecast_input_df=c2_population_df.merge(c2_gdp_pc_df,how='left',on=['year'])
c2_forecast_input_df=c2_forecast_input_df.merge(c2_ei_df,how='left',on=['year'])
c2_forecast_input_df=c2_forecast_input_df.merge(c2_manufacturing_percent_df,how='left',on=['year'])
c2_forecast_input_df=c2_forecast_input_df.merge(c2_trade_df,how='left',on=['year'])
c2_forecast_input_df=c2_forecast_input_df.merge(c2_renew_percent_df,how='left',on=['year'])
c2_forecast_input_df['cluster']='cluster 2'
c2_forecast_input_df.drop(['year'],axis=1,inplace=True)
c2_forecast_input_df['year']=[2021,2022,2023,2024,2025]
c2_forecast_input_df['population_lag4']=c2_forecast_input_df['population'].shift(4)
c2_forecast_input_df['constant_gdp_per_capita_lag4']=c2_forecast_input_df['constant_gdp_per_capita'].shift(4)
c2_forecast_input_df['energy_intensity_lag4']=c2_forecast_input_df['energy_intensity'].shift(4)
c2_forecast_input_df['manufacturing_percent_lag4']=c2_forecast_input_df['manufacturing_percent'].shift(4)
c2_forecast_input_df['trade_openness_lag4']=c2_forecast_input_df['trade_openness'].shift(4)
c2_forecast_input_df['renewable_energy_consumption_share_lag4']=c2_forecast_input_df['renewable_energy_consumption_share'].shift(4)
c2_forecast_input_df['population_growth']=c2_forecast_input_df['population']/c2_forecast_input_df['population_lag4']
c2_forecast_input_df['constant_gdp_per_capita_growth']=c2_forecast_input_df['constant_gdp_per_capita']/c2_forecast_input_df['constant_gdp_per_capita_lag4']
c2_forecast_input_df['energy_intensity_growth']=c2_forecast_input_df['energy_intensity']/c2_forecast_input_df['energy_intensity_lag4']
c2_forecast_input_df['manufacturing_percent_growth']=c2_forecast_input_df['manufacturing_percent']/c2_forecast_input_df['manufacturing_percent_lag4']
c2_forecast_input_df['trade_openness_growth']=c2_forecast_input_df['trade_openness']/c2_forecast_input_df['trade_openness_lag4']
c2_forecast_input_df['renewable_energy_consumption_share_growth']=c2_forecast_input_df['renewable_energy_consumption_share']/c2_forecast_input_df['renewable_energy_consumption_share_lag4']

c2_population_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'population_growth']
c2_constant_gdp_per_capita_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'constant_gdp_per_capita_growth']
c2_energy_intensity_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'energy_intensity_growth']
c2_manufacturing_percent_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'manufacturing_percent_growth']
c2_trade_openness_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'trade_openness_growth']
c2_renewable_energy_consumption_share_growth=c2_forecast_input_df.loc[c2_forecast_input_df.index[(c2_forecast_input_df['year']==2025)][0],'renewable_energy_consumption_share_growth']

#print(c0_population_growth,c0_constant_gdp_per_capita_growth,c0_energy_intensity_growth,c0_manufacturing_percent_growth,c0_trade_openness_growth,c0_renewable_energy_consumption_share_growth)
#print(c1_population_growth,c1_constant_gdp_per_capita_growth,c1_energy_intensity_growth,c1_manufacturing_percent_growth,c1_trade_openness_growth,c1_renewable_energy_consumption_share_growth)
#print(c2_population_growth,c2_constant_gdp_per_capita_growth,c2_energy_intensity_growth,c2_manufacturing_percent_growth,c2_trade_openness_growth,c2_renewable_energy_consumption_share_growth)

In [28]:
# Prepare matrix X to derive the predicted values
analysis_2000_df3=analysis_df[(analysis_df['year']>=1971)&(analysis_df['year']<=2020)&(analysis_df['iso_code']!='WLD')]
analysis_2000_df3 = analysis_2000_df3.dropna(subset=['log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                                                   'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
#                                                  'log_patent_percent',
                                                  ])
co2_pc_exog_vars_hand=['iso_code','year','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                       'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq'
                      ]
features_use=['iso_code','year','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
              'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
              'log_total_co2','log_co2_per_capita','log_co2_intensity']
predict_hand_df3=analysis_2000_df3.copy(deep=True)
predict_hand_df3=predict_hand_df3.dropna(subset=['log_total_co2','log_population','log_urban_percent','log_gdp_per_capita','log_gdp_per_capita_sq','log_energy_intensity',
                              'log_manufacturing_share','log_trade','log_renewable_share','log_renewable_share_sq',
                              ])
exog_hand3=predict_hand_df3[co2_pc_exog_vars_hand]
country_lst=list(exog_hand3['iso_code'].unique())[:]

for i in range(1971,2021):
    exog_hand3[i]=np.where(exog_hand3['year']==i,1,0)
for country in country_lst:
    exog_hand3[country]=np.where(exog_hand3['iso_code']==country,1,0)
exog_hand3.columns=exog_hand3.columns.astype(str)    
d1=exog_hand3[exog_hand3['2020']==1]
d1['cluster']=np.where(d1['iso_code'].isin(country_gp0),'cluster 0',0)
d1['cluster']=np.where(d1['iso_code'].isin(country_gp1),'cluster 1',d1['cluster'])
d1['cluster']=np.where(d1['iso_code'].isin(country_gp2),'cluster 2',d1['cluster'])
d1=d1.reset_index()
d1.drop(['index','year'],axis=1,inplace=True)

dict0={'log_population':c0_population_growth,'log_urban_percent':1,'log_gdp_per_capita':c0_constant_gdp_per_capita_growth,'log_energy_intensity':c0_energy_intensity_growth,'log_manufacturing_share':c0_manufacturing_percent_growth,'log_trade':c0_trade_openness_growth,'log_renewable_share':c0_renewable_energy_consumption_share_growth}
dict1={'log_population':c1_population_growth,'log_urban_percent':1,'log_gdp_per_capita':c1_constant_gdp_per_capita_growth,'log_energy_intensity':c1_energy_intensity_growth,'log_manufacturing_share':c1_manufacturing_percent_growth,'log_trade':c1_trade_openness_growth,'log_renewable_share':c1_renewable_energy_consumption_share_growth}
dict2={'log_population':c2_population_growth,'log_urban_percent':1,'log_gdp_per_capita':c2_constant_gdp_per_capita_growth,'log_energy_intensity':c2_energy_intensity_growth,'log_manufacturing_share':c2_manufacturing_percent_growth,'log_trade':c2_trade_openness_growth,'log_renewable_share':c2_renewable_energy_consumption_share_growth}
feature_list=['log_population','log_urban_percent','log_gdp_per_capita','log_energy_intensity','log_manufacturing_share','log_trade','log_renewable_share']

d1['2025']=0
for name in feature_list:
    for i in range(113):
        if d1.loc[i,'cluster']=='cluster 0':
            d1.loc[i+113,name]=np.log(np.exp(d1.loc[i,name])*dict0[name])
            d1.loc[i+113,'cluster']='cluster 0'
        elif d1.loc[i,'cluster']=='cluster 1':
            d1.loc[i+113,name]=np.log(np.exp(d1.loc[i,name])*dict1[name])
            d1.loc[i+113,'cluster']='cluster 1'
        else:
            d1.loc[i+113,name]=np.log(np.exp(d1.loc[i,name])*dict2[name])
            d1.loc[i+113,'cluster']='cluster 2'
        iso_code_temp=d1.loc[i,'iso_code']
        d1.loc[i+113,iso_code_temp]=1
        d1.loc[i+113,'2025']=1
        d1.loc[i+113,'iso_code']=iso_code_temp
d1['log_gdp_per_capita_sq']=d1['log_gdp_per_capita']**2
d1['log_renewable_share_sq']=d1['log_renewable_share']**2
d1=d1.fillna(0)
d1=d1.tail(113)
predict_result_df1=d1[['iso_code','cluster']].reset_index()
predict_result_df1.drop(['index'],axis=1,inplace=True)
d1.drop(['iso_code','cluster'],axis=1,inplace=True)
col_remove_list=list(range(1971, 2021))
str_col_remove_list = [str(int) for int in col_remove_list]
d1.drop(str_col_remove_list,axis=1,inplace=True)
col_list=d1.columns.tolist()
d1['constant']=1
# Convert to array
d1_array = d1.to_numpy()
#d1_array.shape

In [29]:
# Prepare beta to derive the predicted value
#sum_param_2025=0
#for i in range(50):
#    sum_param_2025=sum_param_2025+(mod_2000_co2_pc_res.params[[i+10]][0])
#param_2025=sum_param_2025/50
beta_list=[]
for i in range(9):
    beta_list.append(mod_2000_co2_pc_res.params[[i+1]][0])
for i in range(160):
    beta_list.append(mod_2000_co2_pc_res.params[[i+56]][0])
beta_list.append(param_2025)
beta_list.append(mod_2000_co2_pc_res.params[[0]][0])
beta_array=np.array(beta_list)
#beta_array.shape

In [30]:
# y = Xb Calculate predicted log total Co2
y_hat_array=np.exp(np.matmul(d1_array,beta_array))
#y_hat_array

In [31]:
# Predict 2025 total CO2 emission using the panel regression model (using individual cluster figures) - total
#(Note: The figure is divided by 1000 to convert the unit from thousand tonne to million tonne for comparison with the time series forecast)
predict_result_df2=pd.DataFrame(y_hat_array,columns=['predict_total_co2'])
predict_result_df = pd.concat([predict_result_df1.reset_index(drop=True),predict_result_df2.reset_index(drop=True)], axis=1)
predict_total_co2 = (predict_result_df['predict_total_co2'].sum())/1000
predict_total_co2

38287.7434318615

In [32]:
# Predict 2025 total CO2 emission using the panel regression model (using individual cluster figures) - individual cluster
predict_cluster_co2 = predict_result_df.groupby(['cluster']).agg({'predict_total_co2':'sum'})
predict_cluster_co2['predict_total_co2']=predict_cluster_co2['predict_total_co2']/1000
predict_cluster_co2=predict_cluster_co2.reset_index()
predict_cluster_co2

,cluster,predict_total_co2
0,cluster 0,789.301573
1,cluster 1,20319.553445
2,cluster 2,17178.888413


In [33]:
all_population_df = pd.DataFrame(all_population_array,columns=['year','population'])
all_population_df['year'].astype(int)
all_gdp_pc_df = pd.DataFrame(all_gdp_pc_array,columns=['year','constant_gdp_per_capita'])
all_gdp_pc_df['year'].astype(int)
all_ei_df = pd.DataFrame(all_ei_array,columns=['year','energy_intensity'])
all_ei_df['year'].astype(int)
all_manufacturing_percent_df = pd.DataFrame(all_manufacturing_percent_array,columns=['year','manufacturing_percent'])
all_manufacturing_percent_df['year'].astype(int)
all_trade_df = pd.DataFrame(all_trade_array,columns=['year','trade_openness'])
all_trade_df['year'].astype(int)
all_renew_percent_df = pd.DataFrame(all_renew_percent_array,columns=['year','renewable_energy_consumption_share'])
all_renew_percent_df['year'].astype(int)
all_forecast_input_df=all_population_df.merge(all_gdp_pc_df,how='left',on=['year'])
all_forecast_input_df=all_forecast_input_df.merge(all_ei_df,how='left',on=['year'])
all_forecast_input_df=all_forecast_input_df.merge(all_manufacturing_percent_df,how='left',on=['year'])
all_forecast_input_df=all_forecast_input_df.merge(all_trade_df,how='left',on=['year'])
all_forecast_input_df=all_forecast_input_df.merge(all_renew_percent_df,how='left',on=['year'])
all_forecast_input_df.drop(['year'],axis=1,inplace=True)
all_forecast_input_df['year']=[2021,2022,2023,2024,2025]
all_forecast_input_df['population_lag4']=all_forecast_input_df['population'].shift(4)
all_forecast_input_df['constant_gdp_per_capita_lag4']=all_forecast_input_df['constant_gdp_per_capita'].shift(4)
all_forecast_input_df['energy_intensity_lag4']=all_forecast_input_df['energy_intensity'].shift(4)
all_forecast_input_df['manufacturing_percent_lag4']=all_forecast_input_df['manufacturing_percent'].shift(4)
all_forecast_input_df['trade_openness_lag4']=all_forecast_input_df['trade_openness'].shift(4)
all_forecast_input_df['renewable_energy_consumption_share_lag4']=all_forecast_input_df['renewable_energy_consumption_share'].shift(4)
all_forecast_input_df['population_growth']=all_forecast_input_df['population']/all_forecast_input_df['population_lag4']
all_forecast_input_df['constant_gdp_per_capita_growth']=all_forecast_input_df['constant_gdp_per_capita']/all_forecast_input_df['constant_gdp_per_capita_lag4']
all_forecast_input_df['energy_intensity_growth']=all_forecast_input_df['energy_intensity']/all_forecast_input_df['energy_intensity_lag4']
all_forecast_input_df['manufacturing_percent_growth']=all_forecast_input_df['manufacturing_percent']/all_forecast_input_df['manufacturing_percent_lag4']
all_forecast_input_df['trade_openness_growth']=all_forecast_input_df['trade_openness']/all_forecast_input_df['trade_openness_lag4']
all_forecast_input_df['renewable_energy_consumption_share_growth']=all_forecast_input_df['renewable_energy_consumption_share']/all_forecast_input_df['renewable_energy_consumption_share_lag4']

all_population_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'population_growth']
all_constant_gdp_per_capita_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'constant_gdp_per_capita_growth']
all_energy_intensity_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'energy_intensity_growth']
all_manufacturing_percent_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'manufacturing_percent_growth']
all_trade_openness_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'trade_openness_growth']
all_renewable_energy_consumption_share_growth=all_forecast_input_df.loc[all_forecast_input_df.index[(all_forecast_input_df['year']==2025)][0],'renewable_energy_consumption_share_growth']

all_population_growth=all_population_growth*population_growth_2016_2020
all_constant_gdp_per_capita_growth=all_constant_gdp_per_capita_growth*gdp_per_capita_growth_2016_2020
all_energy_intensity_growth=all_energy_intensity_growth*energy_intensity_growth_2016_2020
all_manufacturing_percent_growth=all_manufacturing_percent_growth*manufacturing_percent_growth_2016_2020
all_trade_openness_growth=all_trade_openness_growth*trade_openness_growth_2016_2020
all_renewable_energy_consumption_share_growth=all_renewable_energy_consumption_share_growth*renewable_share_growth_2016_2020

#print(all_population_growth,all_constant_gdp_per_capita_growth,all_energy_intensity_growth,all_manufacturing_percent_growth,all_trade_openness_growth,all_renewable_energy_consumption_share_growth)

In [34]:
population_2025=all_population_array[4][1]
urban_population_percent_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'urban_population_percent']*urban_percent_growth_2016_2020
constant_gdp_per_capita_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'constant_gdp_per_capita']*all_constant_gdp_per_capita_growth
energy_intensity_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'energy_intensity']*all_energy_intensity_growth
manufacturing_percent_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'manufacturing_percent']*all_manufacturing_percent_growth
trade_openness_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'trade_openness']*all_trade_openness_growth
renewable_energy_share_2025=data2.loc[data2.index[(data2['year']==2016)&(data2['iso_code']=='WLD')][0],'renewable_energy_consumption_share']*all_renewable_energy_consumption_share_growth

ln_population_2025=np.log(population_2025)
ln_urban_population_percent_2025=np.log(urban_population_percent_2025)
ln_constant_gdp_per_capita_2025=np.log(constant_gdp_per_capita_2025)
ln_constant_gdp_per_capita_sq_2025=np.log(constant_gdp_per_capita_2025)**2
ln_energy_intensity_2025=np.log(energy_intensity_2025)
ln_manufacturing_percent_2025=np.log(manufacturing_percent_2025)
ln_trade_openness_2025=np.log(trade_openness_2025)
ln_renewable_energy_share_2025=np.log(renewable_energy_share_2025)
ln_renewable_energy_share_sq_2025=np.log(renewable_energy_share_2025)**2

X_all_array=np.array([ln_population_2025,ln_urban_population_percent_2025,ln_constant_gdp_per_capita_2025,ln_constant_gdp_per_capita_sq_2025,ln_energy_intensity_2025,ln_manufacturing_percent_2025,ln_trade_openness_2025,ln_renewable_energy_share_2025,ln_renewable_energy_share_sq_2025,1,1,1])
#X_all_array.shape

In [35]:
# Predict 2025 total CO2 emission using the panel regression model (using global CO2 emission figure)
y_hat_all_array=np.exp(np.matmul(np.transpose(X_all_array),beta_all_array))/1000
y_hat_all_array

41371.569199715785

### Summary of prediction - by cluster

In [36]:
predict_2025_total={'cluster':['all'],'predict_total_co2':[predict_total_co2]}
predict_2025_total_df=pd.DataFrame.from_dict(predict_2025_total)
predict_cluster_co2=predict_cluster_co2.append(predict_2025_total_df)
predict_cluster_co2['year']=2025
predict_cluster_co2=predict_cluster_co2[['year','cluster','predict_total_co2']]

In [37]:
predict_test_cluster_co2=predict_test_cluster_co2.append(predict_cluster_co2).reset_index()
predict_test_cluster_co2.drop(['index'],axis=1,inplace=True)
predict_test_cluster_co2

,year,cluster,predict_total_co2,co2
0,2017,cluster 0,892.477209,1345.464
1,2017,cluster 1,19252.443276,15128.210
2,2017,cluster 2,19714.499549,18205.638
3,2017,all,39859.420035,34679.312
4,2018,cluster 0,937.088876,1383.207
5,2018,cluster 1,19976.983043,15665.481
6,2018,cluster 2,20145.781957,18318.256
7,2018,all,41059.853875,35366.944
8,2019,cluster 0,942.161952,1482.379
9,2019,cluster 1,20541.530551,15745.805


### Summary of prediction - global

In [38]:
predict_2025_total_global={'year':[2025],'predict_total_co2':[y_hat_all_array]}
predict_2025_total_global_df=pd.DataFrame.from_dict(predict_2025_total_global)
predict_test_global_co2_df=predict_test_global_co2_df.append(predict_2025_total_global_df).reset_index()
predict_test_global_co2_df.drop(['index'],axis=1,inplace=True)
predict_test_global_co2_df

,year,predict_total_co2,co2
0,2017,41419.521643,35925.738
1,2018,42479.081443,36646.140
2,2019,43062.141679,36702.503
3,2020,41810.826522,34807.259
4,2025,41371.569200,NaN
